In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
xyz = np.loadtxt("data/bathymetry.txt")
xyz.shape
X = xyz[0, :].reshape((98, 135))
Y = xyz[1, :].reshape((98, 135))
zi = xyz[2, :].reshape((98, 135))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax.contourf(X, Y, zi, cmap="Greys", alpha=0.1)
zi_max = -2
zi_masked = np.ma.masked_where(zi > zi_max, zi)
ax.pcolor(X, Y, zi_masked)


In [ ]:
# read the data from excel using pandas
df = pd.read_excel('data/water_balance_data.xlsx',
                   index_col=0,
                   parse_dates=True)
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

cf = ax.contourf(X, Y, zi, cmap="Greys", alpha=0.1)
pc = ax.pcolor(X, Y, zi)

def update_plot(df_index):
    global X, Y, zi, pc, wl_min, wl_max

    wl_i = df.loc[df_index, 'wl']
    
    pc.remove()
    zi_masked = np.ma.masked_where(zi > wl_i, zi)
    pc = ax.pcolor(X, Y, zi_masked, vmin=wl_min, vmax=wl_max)

wl_min = np.nanmin(zi)
wl_max = -0.2
# levels = np.arange(-2.2, -.2, 0.02)

ani = animation.FuncAnimation(
    fig, 
    update_plot, 
    frames=df.index, 
    repeat=True,
)

# Save the animation as an animated gif file
ani.save("bathymetry_simple.gif")

In [ ]:
fig, (ax0, ax1) = plt.subplots(
    nrows=2, 
    figsize=(10, 8),
    gridspec_kw={'height_ratios': [1, 4]},
)

ax2 = plt.axes([0.175, 0.15, 0.2, 0.2])

wl_0 = df["wl"].iloc[0]

ax0.plot(df.index, df["wl"])
hl = ax0.axhline(wl_0, color='k')
vl = ax0.axvline(df.index[0], color='k')

ax0.set_ylim(-1.4, -0.4)
ax0.set_ylabel("Water level")

row = 70
x_xs = X[row]
z_xs = zi[row]
ax2.plot(x_xs, z_xs, 'k')
ax1.axhline(Y[row, 0])
fb = ax2.fill_between(x_xs[z_xs <= wl_0], z_xs[z_xs <= wl_0], wl_0)
ax2.set_xticks([])

cf = ax1.contourf(X, Y, zi, cmap="Greys", alpha=0.1)
pc = ax1.pcolor(X, Y, zi)

def update_plot(df_index):
    global X, Y, zi, pc, hl, vl, fb, df, wl_min, wl_max
    
    wl_i = df.loc[df_index, 'wl']
    
    hl.remove()
    hl = ax0.axhline(wl_i, color='lightgray', lw=1)

    vl.remove()
    vl = ax0.axvline(df_index, color='lightgray', lw=1)

    fb.remove()
    fb = ax2.fill_between(x_xs[z_xs <= wl_i], z_xs[z_xs <= wl_i], wl_i, color='C0')

    pc.remove()
    zi_masked = np.ma.masked_where(zi > wl_i, zi)
    pc = ax1.pcolor(X, Y, zi_masked, vmin=wl_min, vmax=wl_max)


wl_min = np.nanmin(zi)
wl_max = -0.2
# levels = np.arange(-2.2, -.2, 0.02)

ani = animation.FuncAnimation(
    fig, 
    update_plot, 
    frames=df['wl'].index, 
    repeat=True,
)

# Save the animation as an animated gif file
ani.save("bathymetry_fancier.gif")